<a href="https://colab.research.google.com/github/HNXJ/PredictiveCodingNetwork/blob/main/PredictiveLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np


# GPU config if needed
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
# TB
# dir(tf.keras.layers.LSTM)

In [4]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

def create_serial_dataset(x=None, y=None, n=100, length=10, frames=10):

    X = np.zeros([n, length*frames, x.shape[1], x.shape[2]])
    Y = np.zeros([n, length*frames, 10])

    for i in range(n):

        k = np.random.randint(0, 1000, size=(length))
        for j in range(length):
          
            X[i, j*frames:j*frames+frames, :, :] = x[k[j], :, :]
            Y[i, j*frames:j*frames+frames, y[k[j]]] = 1
          
    return X.reshape(n, length*frames, x.shape[1] * x.shape[2]), Y

Xn, Yn = create_serial_dataset(x_train, y_train, n=100, length=10, frames=4)
X = Xn[:10, :, :]
Y = Yn[:10, :, :]
X.shape, Y.shape

11493376/11490434 [==============================] - 0s 0us/step


((10, 40, 784), (10, 40, 10))

In [5]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(None, 784)))
model.add(tf.keras.layers.LSTM(
  units=512,
  recurrent_dropout=0.2,
  return_sequences=True,
  # return_state=True
))
model.add(tf.keras.layers.Dense(units=10))
model.compile(
  loss='mean_squared_error',
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.003)
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 512)         2656256   
_________________________________________________________________
dense (Dense)                (None, None, 10)          5130      
Total params: 2,661,386
Trainable params: 2,661,386
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(
    x=X, y=Y,
    epochs=60,
    batch_size=5,
    validation_split=0.0,
    verbose=1,
    shuffle=False
)

Epoch 1/60
2/2 [==============================] - 4s 74ms/step - loss: 0.6463
Epoch 2/60
2/2 [==============================] - 0s 66ms/step - loss: 0.3347
Epoch 3/60
2/2 [==============================] - 0s 59ms/step - loss: 0.1869
Epoch 4/60
2/2 [==============================] - 0s 58ms/step - loss: 0.1172
Epoch 5/60
2/2 [==============================] - 0s 56ms/step - loss: 0.0934
Epoch 6/60
2/2 [==============================] - 0s 56ms/step - loss: 0.0713
Epoch 7/60
2/2 [==============================] - 0s 56ms/step - loss: 0.0682
Epoch 8/60
2/2 [==============================] - 0s 56ms/step - loss: 0.0642
Epoch 9/60
2/2 [==============================] - 0s 57ms/step - loss: 0.0591
Epoch 10/60
2/2 [==============================] - 0s 56ms/step - loss: 0.0529
Epoch 11/60
2/2 [==============================] - 0s 56ms/step - loss: 0.0471
Epoch 12/60
2/2 [==============================] - 0s 56ms/step - loss: 0.0455
Epoch 13/60
2/2 [==============================] - 0s 57ms/st

In [7]:
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K

# with a Sequential model
L1 = K.function([model.layers[0].input],
                                  [model.layers[0].output])


In [11]:
means1 = []

for k in range(10):
    l, r = k, k+1
    l_pred = L1(Xn[l:r, :, :])[0]
    y_pred = model.predict(Xn[l:r, :, :])

    # for i in range(7):
        # plt.plot(l_pred[0][0, :, i])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means1.append(np.mean(np.mean(np.abs(l_pred))))

In [12]:
means2 = []

for k in range(20, 30):
    l, r = k, k+1
    l_pred = L1(Xn[l:r, :, :])[0]
    y_pred = model.predict(Xn[l:r, :, :])

    # for i in range(7):
        # plt.plot(l_pred[0][0, :, i])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means2.append(np.mean(np.mean(np.abs(l_pred))))

In [13]:
print(np.mean(means1), np.mean(means2))

0.14462271 0.14411935
